In [5]:
import pandas as pd
import numpy as np 
from sklearn.model_selection import train_test_split

## data for train

In [60]:

def get_data(fname, isTrain=True):
    data = []
    with open(fname, 'r', encoding='utf-8') as f:
        for line in f:
            data.append(line) 

    train_dict = {}

    if isTrain == True:
        for data_str in data:
            item = eval(data_str)
            train_dict[item["id"]] = {"media" : item["media"], 
                                     "article_original" : item["article_original"],
                                     "abs":item["abstractive"],
                                     "ext":item["extractive"]}
    else:
        for data_str in data:
            item = eval(data_str)
            train_dict[item["id"]] = {"media" : item["media"], 
                                     "article_original" : item["article_original"]}
        
    return train_dict

In [26]:
data_dict={}

for item in train_dict.items():
    
    for idx_, cont in enumerate(item[1]["article_original"]):

        data_id =  str(item[0]) + str(idx_).zfill(3)

        if idx_ in item[1]["ext"]:
            label = 1
        else:
            label = 0
        
        data_dict[data_id]={"document" :cont,
                           "label" : label}

In [32]:
total_data = pd.DataFrame(data_dict)
total_data = total_data.transpose()
total_data = total_data.reset_index()
total_data = total_data.rename(columns={"index" :"id"})

In [51]:
X_train, X_test = train_test_split(total_data, test_size=0.2, random_state=123, stratify=total_data["label"])

X_train.to_csv("/home/miyoun/sangwon.joo/dacon/KoBERT/dacon_ext_summary/data/dacon_train.txt", sep="\t", header=None, index=False)
X_test.to_csv("/home/miyoun/sangwon.joo/dacon/KoBERT/dacon_ext_summary/data/dacon_test.txt", sep="\t", header=None, index=False)

## data for submission

In [61]:
fname = "/home/miyoun/sangwon.joo/dacon/KoBERT/dacon_ext_summary/data/extractive_test_v2.jsonl"
data = get_data(fname, False)

In [62]:
data

{'500733466': {'media': '당진시대',
  'article_original': ['▲ 석문간척지 임차법인협의회가 한국농어촌공사 당진지사 앞에 공공비축벼 320t을 쌓아두고 시위를 벌이고 있다.',
   '석문간척지 임차법인협의회(이하 간척지협의회)가 농림축산식품부의 부당한 간척지 임대료 책정에 반발하며 지난달 30일 한국농어촌공사 당진지사 앞에 공공비축벼 320t을 쌓고 시위를 벌였다.',
   '43개 영농조합법인이 소속된 간척지협의회는 이번 벼 야적 시위를 통해 현재 1kg당 2100원으로 책정된 임대료를 현재 쌀 판매가격인 1300원대로 인하할 것을 요구하고 있다.',
   '이들은 지난 12월 7일 농림축산식품부에 탄원서를 제출했지만 “임대료 인하는 올해 이후에나 가능하다”고 통보받은 상황이다.',
   '게다가 임차법인들의 계약기간이 올해 만료되기 때문에 임대료를 인하해도 지난 2년 동안의 손실 보상은 받을 수 없는 상황이다.',
   '이에 간척지협의회는 계약기간을 2년 연장하고, 연장된 기간 동안 인하된 임대료를 적용해 지난 2년 간의 손실에 대해 보상할 것을 제안했다.',
   '더불어 요구사항이 받아들여지지 않을 경우 벼 야적시위를 시작한 날짜인 지난해 12월 30일자로 임대료를 벼로 납부하겠다는 입장이다.',
   '김재용 봉치영농조합법인 조합원은 “현재 한국농어촌공사의 답변을 기다리고 있다”며 “상황을 지켜본 뒤 추가적인 야적 시위 여부를 결정할 계획”이라고 말했다.',
   '이어 “법을 만들고 집행할 때 현실성 있게 만들어야 한다”며 “농민이 정부를 믿을 수 있도록 조속히 해결책을 마련해주길 바란다”고 덧붙였다.']},
 '500742482': {'media': '당진시대',
  'article_original': ['신 벌떼해장국이 손님들의 성원에 보답하고자 24시간 영업을 재개한다.',
   '또한 이와 함께 오는 12일부터 31일까지 특별 감사 이벤트를 실시할 예정이다.',
   '이 기간 동안 야간(저녁 11시~

In [71]:
data_dict={}

for item in data.items():
    
    for idx_, cont in enumerate(item[1]["article_original"]):

        data_id =  str(item[0]) + str(idx_).zfill(3)
        
        data_dict[data_id]={"document" :cont,
                           "label" : data_id}

In [72]:
total_data = pd.DataFrame(data_dict)
total_data = total_data.transpose()
total_data = total_data.reset_index()
total_data = total_data.rename(columns={"index" :"id"})

In [73]:
total_data
total_data.to_csv("/home/miyoun/sangwon.joo/dacon/KoBERT/dacon_ext_summary/data/dacon_submission.txt", sep="\t", header=None, index=False)

In [70]:
len(X_test)

113485

In [78]:
total_data["sent_num"] = total_data["label"].map(lambda x: x[:-3])

In [80]:
total_data["score"] = range(0, len(total_data))

In [89]:
total_data["rank"] = total_data.groupby("sent_num")["score"].rank(method="min", ascending=True)
total_data["imp_sentence"] = total_data["rank"].map(lambda x: 1 if x <=3 else 0)

In [98]:
sent_list = set(list(total_data["sent_num"]))

In [123]:
result_dict={}

for sent in sent_list:
    tmp = total_data[total_data["sent_num"]=="500733466"]
    
    answer = tmp[tmp["rank"]==1]["document"].item() + "\n" + tmp[tmp["rank"]==2]["document"].item() + "\n" + tmp[tmp["rank"]==3]["document"].item()
    print(answer)
    
    result_dict[sent] = answer
    

▲ 석문간척지 임차법인협의회가 한국농어촌공사 당진지사 앞에 공공비축벼 320t을 쌓아두고 시위를 벌이고 있다.
석문간척지 임차법인협의회(이하 간척지협의회)가 농림축산식품부의 부당한 간척지 임대료 책정에 반발하며 지난달 30일 한국농어촌공사 당진지사 앞에 공공비축벼 320t을 쌓고 시위를 벌였다.
43개 영농조합법인이 소속된 간척지협의회는 이번 벼 야적 시위를 통해 현재 1kg당 2100원으로 책정된 임대료를 현재 쌀 판매가격인 1300원대로 인하할 것을 요구하고 있다.


/home/miyoun/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: FutureWarning: `item` has been deprecated and will be removed in a future version
  


In [124]:
result_dict

{'678547366': '▲ 석문간척지 임차법인협의회가 한국농어촌공사 당진지사 앞에 공공비축벼 320t을 쌓아두고 시위를 벌이고 있다.\n석문간척지 임차법인협의회(이하 간척지협의회)가 농림축산식품부의 부당한 간척지 임대료 책정에 반발하며 지난달 30일 한국농어촌공사 당진지사 앞에 공공비축벼 320t을 쌓고 시위를 벌였다.\n43개 영농조합법인이 소속된 간척지협의회는 이번 벼 야적 시위를 통해 현재 1kg당 2100원으로 책정된 임대료를 현재 쌀 판매가격인 1300원대로 인하할 것을 요구하고 있다.'}

In [127]:
result_dict={}

for sent in sent_list:
    tmp = total_data[total_data["sent_num"]=="500733466"]
    
    print("\n".join(list(tmp[tmp["imp_sentence"]==1]["document"])))
    
    result_dict[sent] = answer
    break

▲ 석문간척지 임차법인협의회가 한국농어촌공사 당진지사 앞에 공공비축벼 320t을 쌓아두고 시위를 벌이고 있다.
석문간척지 임차법인협의회(이하 간척지협의회)가 농림축산식품부의 부당한 간척지 임대료 책정에 반발하며 지난달 30일 한국농어촌공사 당진지사 앞에 공공비축벼 320t을 쌓고 시위를 벌였다.
43개 영농조합법인이 소속된 간척지협의회는 이번 벼 야적 시위를 통해 현재 1kg당 2100원으로 책정된 임대료를 현재 쌀 판매가격인 1300원대로 인하할 것을 요구하고 있다.


In [ ]:
list(tmp[tmp["imp_sentence"]==1]["document"]).join("\n")